In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv('../input/homicide-reports/database.csv')


In [ ]:
df.head()

In [ ]:

plt.style.use('fivethirtyeight')

years = pd.DataFrame(df, columns = ['Year']) 
count_years = years.stack().value_counts()
homicides = count_years.sort_index(axis=0, ascending=False)
#plot the total of homicides
print(homicides.plot(kind='barh', fontsize=10,  width=0.5,  figsize=(12, 10), title='Отчеты об убийствах в США 1980-2014 '))


In [ ]:
g = sns.factorplot("Month", data=df, aspect=1.5, kind="count")
g.set_xticklabels(rotation=90)



In [ ]:
df.groupby(["State", "Weapon"]).size().reset_index(name="Count").sort_values(by='Count', ascending=False)

In [ ]:
bins = np.arange(0, 100, 5)
grid = sns.FacetGrid(df, col='Victim Sex', size=3.3, aspect=2)
grid.map(plt.hist,'Victim Age', bins = bins)
grid.add_legend()

In [ ]:
p_null= (len(df) - df.count())*100.0/len(df)
p_null

In [ ]:
train = df[['Year','Incident','Crime Solved','Victim Age','Victim Sex']]
df.isnull().any() 

In [ ]:
train['Crime Solved'].fillna('Yes', inplace = True)
train.isnull().any()

In [ ]:
train['Crime Solved'].interpolate(inplace = True)
train.isnull().any()

In [ ]:
train['Crime Solved'].replace('Yes', 1, inplace = True)
train['Crime Solved'].replace('No', 0, inplace = True)
train['Victim Sex'].replace('Male', 0, inplace = True)
train['Victim Sex'].replace('Female', 1, inplace = True)
train['Victim Sex'].replace('Unknown', 2, inplace = True)

train.head()

In [ ]:
sns.heatmap(train.corr(),cmap='coolwarm',annot=True)

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(train[['Crime Solved']])
train[['Crime Solved']] = pd.DataFrame(scaled)
train.head()

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(train[['Year','Incident','Victim Age','Victim Sex']], train['Crime Solved'], test_size = 0.3)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
drugTree = DecisionTreeClassifier(criterion="gini")
drugTree.fit(X_train,y_train)
predTree = drugTree.predict(X_test)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
pred = knn.predict(X_test)

nbc = GaussianNB()
nbc.fit(X_train,y_train)
y_pred = nbc.predict(X_test)

logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)
predictions = logmodel.predict(X_test)

In [ ]:
from sklearn import metrics

print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, predTree))
print(classification_report(y_test, predTree))
pd.DataFrame(
confusion_matrix(y_test, predTree),
columns=['Predicted No', 'Predicted Yes'],
index=['Actual No', 'Actual Yes']
) 

In [ ]:
print("KNN's Accuracy: ", metrics.accuracy_score(y_test, pred))
print(classification_report(y_test, pred))
pd.DataFrame(
confusion_matrix(y_test, pred),
columns=['Predicted No', 'Predicted Yes'],
index=['Actual No', 'Actual Yes']
)

In [ ]:
print("NB's Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
pd.DataFrame(
confusion_matrix(y_test, y_pred),
columns=['Predicted No', 'Predicted Yes'],
index=['Actual No', 'Actual Yes']
)

In [ ]:
print("LR's Accuracy: ", metrics.accuracy_score(y_test, predictions ))
print(classification_report(y_test, predictions))
pd.DataFrame(
confusion_matrix(y_test, predictions),
columns=['Predicted No', 'Predicted Yes'],
index=['Actual No', 'Actual Yes']
)

In [ ]:
target=df['Crime Solved']
target_count = target.value_counts()
print('Class 0:', target_count[0])
print('Class 1:', target_count[1])
print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')